<h3> Base Code , Polices , Training  </h3>

In [ ]:
import numpy as np
import random
from itertools import combinations

GRID_SIZE = 5
PICKUP_LOCATIONS = {(0, 4): 5, (1, 3): 5, (4, 1): 5}  # Pickup locations with their capacities
DROPOFF_LOCATIONS = {(0, 0): 0, (2, 0): 0, (3, 4): 0}  # Dropoff locations with their remaining capacities
ACTIONS = ['N', 'E', 'S', 'W', 'P', 'D']  # Actions: North, East, South, West, Pickup, Dropoff
ACTION_REWARDS = {'P': 13, 'D': 13, 'N': -1, 'E': -1, 'S': -1, 'W': -1}  # Reward values for actions
random.seed(4)
np.random.seed(4)

# Hyperparameters
LEARNING_RATE = 0.1
DISCOUNT_FACTOR = 0.95
EPSILON = 0.99
EPISODES = 1000

# Initial positions for each agent
initial_positions = {'red': (2, 2), 'blue': (4, 2), 'black': (0, 2)}  # Adjusted to 0-based indexing

# Agent class definition
class Agent:
    def __init__(self, start_position, name):
        self.position = start_position
        self.name = name
        self.carrying = False
        self.q_table = np.zeros((GRID_SIZE, GRID_SIZE, len(ACTIONS)))

    def get_valid_actions(self):
        x, y = self.position
        valid_actions = ACTIONS.copy()

        # Check if agent is at a border
        if x == 0: valid_actions.remove('N')
        if x == GRID_SIZE - 1: valid_actions.remove('S')
        if y == 0: valid_actions.remove('W')
        if y == GRID_SIZE - 1: valid_actions.remove('E')

        # Check for pickup/dropoff actions
        if (x, y) in PICKUP_LOCATIONS and (PICKUP_LOCATIONS[(x, y)] <= 0 or self.carrying):
            valid_actions.remove('P')
        if (x, y) in DROPOFF_LOCATIONS and (DROPOFF_LOCATIONS[(x, y)] >= 5 or not self.carrying):
            valid_actions.remove('D')
        return valid_actions

    
    def select_action(self, valid_actions, policy='PRandom'):        
        if not valid_actions:
            return None  # No valid actions available
        if policy == 'PRandom' or (policy == 'PExploit' and random.random() < EPSILON):
            return random.choice(valid_actions)
        else:
            q_values = [self.q_table[x, y, ACTIONS.index(a)] for a in valid_actions]
            max_q_value = max(q_values)
            max_actions = [a for a, q in zip(valid_actions, q_values) if q == max_q_value]
            return random.choice(max_actions)

    def perform_action(self, action):  
        x, y = self.position

        # Update position based on action
        if action == 'N': x = max(0, x - 1)
        elif action == 'S': x = min(GRID_SIZE - 1, x + 1)
        elif action == 'E': y = min(GRID_SIZE - 1, y + 1)
        elif action == 'W': y = max(0, y - 1)

        # Handle pickup and dropoff actions
        if action == 'P' and (x, y) in PICKUP_LOCATIONS and PICKUP_LOCATIONS[(x, y)] > 0:
            self.carrying = True
            PICKUP_LOCATIONS[(x, y)] -= 1
        elif action == 'D' and (x, y) in DROPOFF_LOCATIONS and DROPOFF_LOCATIONS[(x, y)] < 5:
            self.carrying = False
            DROPOFF_LOCATIONS[(x, y)] += 1

        self.position = (x, y)
        return self.position

    def update_q_table(self, action, reward, next_state, next_valid_actions):
        if action is None:
            return  # Skip Q-table update if no action was taken
        old_x, old_y = self.position
        new_x, new_y = next_state
        action_index = ACTIONS.index(action)
        future_rewards = [self.q_table[new_x, new_y, ACTIONS.index(a)] for a in next_valid_actions]
        self.q_table[old_x, old_y, action_index] = (1 - LEARNING_RATE) * self.q_table[old_x, old_y, action_index] + \
            LEARNING_RATE * (reward + DISCOUNT_FACTOR * np.max(future_rewards))

    def is_terminal_state(self):
        return all(blocks == 5 for blocks in DROPOFF_LOCATIONS.values())

    def reset(self):
        initial_positions = {'red': (2, 2), 'blue': (4, 2), 'black': (0, 2)}
        self.position = initial_positions[self.name]
        self.carrying = False
        for loc in PICKUP_LOCATIONS:
            PICKUP_LOCATIONS[loc] = 5
        for loc in DROPOFF_LOCATIONS:
            DROPOFF_LOCATIONS[loc] = 0

# Utility functions
def manhattan_distance(pos1, pos2):
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def average_manhattan_distance(agents):
    distances = [manhattan_distance(agent1.position, agent2.position) 
                 for agent1, agent2 in combinations(agents, 2)]
    return np.mean(distances)

def get_shortest_path_length(start, end):
    return abs(start[0] - end[0]) + abs(start[1] - end[1])

def simulate_agent_path(agent, start, end):
    agent.reset()
    agent.position = start
    path = [start]
    while agent.position != end:
        valid_actions = agent.get_valid_actions()
        action = agent.select_action(valid_actions)
        agent.perform_action(action)
        path.append(agent.position)
    return path

# Simulation
agents = [Agent(initial_positions['red'], 'red'), 
          Agent(initial_positions['blue'], 'blue'), 
          Agent(initial_positions['black'], 'black')]

average_distances = []

for episode in range(EPISODES):
    episode_distances = []
    for agent in agents:
        valid_actions = agent.get_valid_actions()
        action = agent.select_action(valid_actions, policy='PRandom')  
        if action:
            new_position = agent.perform_action(action)
            reward = ACTION_REWARDS.get(action, 0)
            next_valid_actions = agent.get_valid_actions()
            agent.update_q_table(action, reward, new_position, next_valid_actions)
        episode_distances.append(average_manhattan_distance(agents))
    average_distances.append(np.mean(episode_distances))
    if agent.is_terminal_state():
        for a in agents:
            a.reset()
        break

# Output analysis
print(f"Average Manhattan distances over episodes:")
for i, avg_dist in enumerate(average_distances, 1):
    print(f"Episode {i}: {avg_dist:.2f}")

# Output Q-tables for analysis
for agent in agents:
    print(f"{agent.name}'s Q-table:")
    print("State  |   S       W       N       E       P       D")
    print("---------------------------------------------------")
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            q_values = agent.q_table[x, y]
            print(f"({x+1},{y+1})  |  " + "  ".join(f"{q_value:6.2f}" for q_value in q_values))
    print("\n")

# Simulate a path from a pickup to a dropoff location
agent = Agent(initial_positions['red'], 'red')  # You can choose any agent here
pickup_location = next(iter(PICKUP_LOCATIONS))
dropoff_location = next(iter(DROPOFF_LOCATIONS))
path = simulate_agent_path(agent, pickup_location, dropoff_location)

# Calculate the shortest path length
shortest_path_len = get_shortest_path_length(pickup_location, dropoff_location)

print(f"Simulated path: {path}")
print(f"Simulated path length: {len(path) - 1}")
print(f"Shortest path length: {shortest_path_len}")

a.	Agent coordination: Do the three agents get in their ways blocking each other or do they do a good job in dividing the transportation task intelligently among one another. Agent coordination could, for example, be measured by computing the average Manhattan distance between the three agents during the run of a specific experiment. You can show the progress of average distance for different iterations. b.	Paths learned: Does the particular approach do a good job in learning paths between block sources and block destinations; is the learnt path the shortest path or close to the shortest path between the source and the destination.  c.	Cost Efficient Learning: Does the particular approach do a good job learning efficient paths or avoiding risky areas. How the risky areas are traversed by the agents as the training goes on and is there any avoidance procedure followed by the trained agents. 

Extra Credit On The Compare   For the task 2 and 3 below





<h3> Experiment 1    Alpha=0.3 Y=0.5   Steps=9000   , PRandom=500 </h3>    

<h3> EXP1 A.PRandom=8500</h3>  

In [3]:
import numpy as np
import random

# Define the environment setup
GRID_SIZE = 5
PICKUP_LOCATIONS = {(0, 4): 5, (1, 3): 5, (4, 1): 5}
DROPOFF_LOCATIONS = {(0, 0): 0, (2, 0): 0, (3, 4): 0}
ACTIONS = ['N', 'E', 'S', 'W', 'P', 'D']
ACTION_REWARDS = {'P': 13, 'D': 13, 'N': -1, 'E': -1, 'S': -1, 'W': -1}

# Hyperparameters
LEARNING_RATE = 0.3
DISCOUNT_FACTOR = 0.5
STEPS_PRANDOM_INITIAL = 500  # Initial random steps
STEPS_PRANDOM_EXTENDED = 8500  # Extended random steps

# Seed for reproducibility
random.seed(6)
np.random.seed(6)

class Agent:
    def __init__(self, start_position, name):
        self.position = start_position
        self.name = name
        self.carrying = False
        self.q_table = np.zeros((GRID_SIZE, GRID_SIZE, len(ACTIONS)))

    def get_valid_actions(self):
        x, y = self.position
        valid_actions = ACTIONS.copy()
        if x == 0: valid_actions.remove('N')
        if x == GRID_SIZE - 1: valid_actions.remove('S')
        if y == 0: valid_actions.remove('W')
        if y == GRID_SIZE - 1: valid_actions.remove('E')
        if self.carrying and (x, y) in DROPOFF_LOCATIONS:
            valid_actions.append('D')
        elif not self.carrying and (x, y) in PICKUP_LOCATIONS and PICKUP_LOCATIONS[(x, y)] > 0:
            valid_actions.append('P')
        return valid_actions

    def select_action(self, valid_actions, policy):
        if not valid_actions:
            return None
        return random.choice(valid_actions)  # PRandom policy selects randomly among valid actions

    def perform_action(self, action):
        if action is None:
            return
        move_delta = {'N': (-1, 0), 'S': (1, 0), 'E': (0, 1), 'W': (0, -1)}
        x, y = self.position
        x += move_delta.get(action, (0, 0))[0]
        y += move_delta.get(action, (0, 0))[1]
        if action == 'P' and (x, y) in PICKUP_LOCATIONS:
            self.carrying = True
            PICKUP_LOCATIONS[(x, y)] -= 1
        elif action == 'D' and (x, y) in DROPOFF_LOCATIONS:
            self.carrying = False
            DROPOFF_LOCATIONS[(x, y)] += 1
        self.position = (x, y)

    def update_q_table(self, action, reward, next_position):
        if action is None:
            return
        old_q = self.q_table[self.position[0], self.position[1], ACTIONS.index(action)]
        next_max_q = np.max(self.q_table[next_position[0], next_position[1]])
        self.q_table[self.position[0], self.position[1], ACTIONS.index(action)] = \
            (1 - LEARNING_RATE) * old_q + LEARNING_RATE * (reward + DISCOUNT_FACTOR * next_max_q)

    def is_terminal_state(self):
        return all(value == 5 for value in DROPOFF_LOCATIONS.values())

    def reset(self):
        self.position = initial_positions[self.name]
        self.carrying = False
        for loc in PICKUP_LOCATIONS:
            PICKUP_LOCATIONS[loc] = 5
        for loc in DROPOFF_LOCATIONS:
            DROPOFF_LOCATIONS[loc] = 0

def run_experiment(agents, policy, steps):
    for step in range(steps):
        for agent in agents:
            valid_actions = agent.get_valid_actions()
            action = agent.select_action(valid_actions, policy)
            agent.perform_action(action)
            reward = ACTION_REWARDS.get(action, 0)
            agent.update_q_table(action, reward, agent.position)
            if agent.is_terminal_state():
                agent.reset()

# Initialize agents
initial_positions = {'red': (2, 2), 'blue': (4, 2), 'black': (0, 2)}
agents = [Agent(initial_positions[name], name) for name in initial_positions]

# Run Experiments
run_experiment(agents, 'PRandom', STEPS_PRANDOM_INITIAL)  # Initial random phase
run_experiment(agents, 'PRandom', STEPS_PRANDOM_EXTENDED)  # Continue with PRandom

# Output results
for agent in agents:
    print(f"{agent.name}'s Q-table after Experiment 1a:")
    print("State  |   N       E       S       W       P       D")
    print("---------------------------------------------------")
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            q_values = agent.q_table[x, y]
            print(f"({x+1},{y+1})  |  " + "  ".join(f"{q_value:6.2f}" for q_value in q_values))
    print("\n")


red's Q-table after Experiment 1a:
State  |   N       E       S       W       P       D
---------------------------------------------------
(1,1)  |   12.00    0.00    0.00   12.00   26.00   26.00
(1,2)  |   12.00   12.00    0.00   12.00   26.00   26.00
(1,3)  |   12.00   12.00    0.00   12.00   26.00   26.00
(1,4)  |   12.00   12.00    0.00   12.00   26.00   26.00
(1,5)  |   12.00   12.00    0.00    0.00   26.00   26.00
(2,1)  |   12.00    0.00   12.00   12.00   26.00   26.00
(2,2)  |   12.00   12.00   12.00   12.00   26.00   26.00
(2,3)  |   12.00   12.00   12.00   12.00   26.00   26.00
(2,4)  |   12.00   12.00   12.00   12.00   26.00   26.00
(2,5)  |   12.00   12.00   12.00    0.00   26.00   26.00
(3,1)  |   12.00    0.00   12.00   12.00   26.00   26.00
(3,2)  |   12.00   12.00   12.00   12.00   26.00   26.00
(3,3)  |   12.00   12.00   12.00   12.00   26.00   26.00
(3,4)  |   12.00   12.00   12.00   12.00   26.00   26.00
(3,5)  |   12.00   12.00   12.00    0.00   26.00   26.00
(4,1)

<h3> EXP1 B.GREEDY=8500</h3> 

In [9]:
import numpy as np
import random

# Environment setup
GRID_SIZE = 5
PICKUP_LOCATIONS = {(0, 4): 5, (1, 3): 5, (4, 1): 5}
DROPOFF_LOCATIONS = {(0, 0): 0, (2, 0): 0, (3, 4): 0}
ACTIONS = ['N', 'E', 'S', 'W', 'P', 'D']
ACTION_REWARDS = {'P': 13, 'D': 13, 'N': -1, 'E': -1, 'S': -1, 'W': -1}

# Hyperparameters
LEARNING_RATE = 0.3
DISCOUNT_FACTOR = 0.5
STEPS_PRANDOM = 500  # Initial random steps
STEPS_PGREEDY = 8500  # Steps for PGreedy policy

# Seed for reproducibility
random.seed(5)
np.random.seed(5)

class Agent:
    def __init__(self, start_position, name):
        self.position = start_position
        self.name = name
        self.carrying = False
        self.q_table = np.zeros((GRID_SIZE, GRID_SIZE, len(ACTIONS)))

    def get_valid_actions(self):
        x, y = self.position
        valid_actions = ACTIONS.copy()
        if x == 0: valid_actions.remove('N')
        if x == GRID_SIZE - 1: valid_actions.remove('S')
        if y == 0: valid_actions.remove('W')
        if y == GRID_SIZE - 1: valid_actions.remove('E')
        if self.carrying and (x, y) in DROPOFF_LOCATIONS:
            valid_actions.append('D')
        elif not self.carrying and (x, y) in PICKUP_LOCATIONS and PICKUP_LOCATIONS[(x, y)] > 0:
            valid_actions.append('P')
        return valid_actions

    def select_action(self, valid_actions, policy):
        if not valid_actions:
            return None
        if policy == 'PGreedy':
            q_values = [self.q_table[x, y, ACTIONS.index(a)] for a in valid_actions]
            max_value = max(q_values)
            best_actions = [a for a, q in zip(valid_actions, q_values) if q == max_value]
            return random.choice(best_actions)

    def perform_action(self, action):
        if action is None:
            return
        move_delta = {'N': (-1, 0), 'S': (1, 0), 'E': (0, 1), 'W': (0, -1)}
        x, y = self.position
        x += move_delta.get(action, (0, 0))[0]
        y += move_delta.get(action, (0, 0))[1]
        if action == 'P' and (x, y) in PICKUP_LOCATIONS:
            self.carrying = True
            PICKUP_LOCATIONS[(x, y)] -= 1
        elif action == 'D' and (x, y) in DROPOFF_LOCATIONS:
            self.carrying = False
            DROPOFF_LOCATIONS[(x, y)] += 1
        self.position = (x, y)

    def update_q_table(self, action, reward, next_position):
        if action is None:
            return
        old_q = self.q_table[self.position[0], self.position[1], ACTIONS.index(action)]
        next_max_q = np.max(self.q_table[next_position[0], next_position[1]])
        self.q_table[self.position[0], self.position[1], ACTIONS.index(action)] = \
            (1 - LEARNING_RATE) * old_q + LEARNING_RATE * (reward + DISCOUNT_FACTOR * next_max_q)

    def is_terminal_state(self):
        return all(value == 5 for value in DROPOFF_LOCATIONS.values())

    def reset(self):
        self.position = initial_positions[self.name]
        self.carrying = False
        for loc in PICKUP_LOCATIONS:
            PICKUP_LOCATIONS[loc] = 5
        for loc in DROPOFF_LOCATIONS:
            DROPOFF_LOCATIONS[loc] = 0

def run_experiment(agents, policy, steps):
    for step in range(steps):
        for agent in agents:
            valid_actions = agent.get_valid_actions()
            action = agent.select_action(valid_actions, policy)
            agent.perform_action(action)
            reward = ACTION_REWARDS.get(action, 0)
            agent.update_q_table(action, reward, agent.position)
            if agent.is_terminal_state():
                agent.reset()

# Initialize agents
initial_positions = {'red': (2, 2), 'blue': (4, 2), 'black': (0, 2)}
agents = [Agent(initial_positions[name], name) for name in initial_positions]

# Run Experiments
run_experiment(agents, 'PRandom', STEPS_PRANDOM)  # Initial random phase
run_experiment(agents, 'PGreedy', STEPS_PGREEDY)  # Experiment 1b

# Output results
for agent in agents:
    print(f"{agent.name}'s Q-table after Experiment 1b:")
    print("State  |   N       E       S       W       P       D")
    print("---------------------------------------------------")
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            q_values = agent.q_table[x, y]
            print(f"({x+1},{y+1})  |  " + "  ".join(f"{q_value:6.2f}" for q_value in q_values))
    print("\n")


red's Q-table after Experiment 1b:
State  |   N       E       S       W       P       D
---------------------------------------------------
(1,1)  |   12.00    0.00    0.00   -0.30   26.00   26.00
(1,2)  |    0.28    0.28    0.00   -0.07    4.48    3.90
(1,3)  |   -0.25   -0.17    0.00    0.16    4.48    3.90
(1,4)  |    0.00   -0.51    0.00    0.00    4.48    7.30
(1,5)  |    0.00    0.00    0.00    0.00    0.00    0.00
(2,1)  |   12.00    0.00   12.00    0.00   26.00   26.00
(2,2)  |    1.40    0.28    0.00    0.99    5.40   10.03
(2,3)  |    1.33    3.02    3.45    0.00    8.91   12.49
(2,4)  |    0.78   -0.30    0.00    0.98    7.21    0.00
(2,5)  |    0.43   -0.51    0.00    0.00    0.00    7.21
(3,1)  |   12.00    0.00   12.00    0.07   26.00   26.00
(3,2)  |    0.00    0.37   -0.30    0.38    3.90    4.48
(3,3)  |    1.77    0.63    2.38    2.47   10.11    8.56
(3,4)  |    1.71    0.98    0.57    1.33    0.00    7.21
(3,5)  |    1.22   -0.30    0.86    0.00   10.11    4.48
(4,1)

<h3> EXP1 C.Exploit =8500</h3> 

In [8]:
import numpy as np
import random

# Define the environment setup
GRID_SIZE = 5
PICKUP_LOCATIONS = {(0, 4): 5, (1, 3): 5, (4, 1): 5}
DROPOFF_LOCATIONS = {(0, 0): 0, (2, 0): 0, (3, 4): 0}
ACTIONS = ['N', 'E', 'S', 'W', 'P', 'D']
ACTION_REWARDS = {'P': 13, 'D': 13, 'N': -1, 'E': -1, 'S': -1, 'W': -1}

# Hyperparameters
LEARNING_RATE = 0.3
DISCOUNT_FACTOR = 0.5
STEPS_PRANDOM = 500  # Initial random steps
STEPS_POLICY = 8500  # Steps for PExploit policy

# Seed for reproducibility
random.seed(4)
np.random.seed(4)

class Agent:
    def __init__(self, start_position, name):
        self.position = start_position
        self.name = name
        self.carrying = False
        self.q_table = np.zeros((GRID_SIZE, GRID_SIZE, len(ACTIONS)))

    def get_valid_actions(self):
        x, y = self.position
        valid_actions = ACTIONS.copy()
        if x == 0: valid_actions.remove('N')
        if x == GRID_SIZE - 1: valid_actions.remove('S')
        if y == 0: valid_actions.remove('W')
        if y == GRID_SIZE - 1: valid_actions.remove('E')
        if self.carrying and (x, y) in DROPOFF_LOCATIONS:
            valid_actions.append('D')
        elif not self.carrying and (x, y) in PICKUP_LOCATIONS and PICKUP_LOCATIONS[(x, y)] > 0:
            valid_actions.append('P')
        return valid_actions

    def select_action(self, valid_actions, policy):
        if not valid_actions:
            return None
        if policy == 'PRandom':
            return random.choice(valid_actions)
        elif policy == 'PExploit':
            q_values = [self.q_table[x, y, ACTIONS.index(a)] for a in valid_actions]
            max_value = max(q_values)
            best_actions = [a for a, q in zip(valid_actions, q_values) if q == max_value]
            return random.choice(best_actions)

    def perform_action(self, action):
        if action is None:
            return
        move_delta = {'N': (-1, 0), 'S': (1, 0), 'E': (0, 1), 'W': (0, -1)}
        x, y = self.position
        x += move_delta.get(action, (0, 0))[0]
        y += move_delta.get(action, (0, 0))[1]
        if action == 'P' and (x, y) in PICKUP_LOCATIONS:
            self.carrying = True
            PICKUP_LOCATIONS[(x, y)] -= 1
        elif action == 'D' and (x, y) in DROPOFF_LOCATIONS:
            self.carrying = False
            DROPOFF_LOCATIONS[(x, y)] += 1
        self.position = (x, y)

    def update_q_table(self, action, reward, next_position):
        if action is None:
            return
        old_q = self.q_table[self.position[0], self.position[1], ACTIONS.index(action)]
        next_max_q = np.max(self.q_table[next_position[0], next_position[1]])
        self.q_table[self.position[0], self.position[1], ACTIONS.index(action)] = \
            (1 - LEARNING_RATE) * old_q + LEARNING_RATE * (reward + DISCOUNT_FACTOR * next_max_q)

    def is_terminal_state(self):
        return all(value == 5 for value in DROPOFF_LOCATIONS.values())

    def reset(self):
        self.position = initial_positions[self.name]
        self.carrying = False
        for loc in PICKUP_LOCATIONS:
            PICKUP_LOCATIONS[loc] = 5
        for loc in DROPOFF_LOCATIONS:
            DROPOFF_LOCATIONS[loc] = 0

initial_positions = {'red': (2, 2), 'blue': (4, 2), 'black': (0, 2)}
agents = [Agent(initial_positions[name], name) for name in initial_positions]

def run_experiment(agents, policy, steps):
    for step in range(steps):
        for agent in agents:
            valid_actions = agent.get_valid_actions()
            action = agent.select_action(valid_actions, policy)
            agent.perform_action(action)
            reward = ACTION_REWARDS.get(action, 0)
            agent.update_q_table(action, reward, agent.position)
            if agent.is_terminal_state():
                agent.reset()

# Run Experiments
run_experiment(agents, 'PRandom', STEPS_PRANDOM)  # Initial random phase
run_experiment(agents, 'PExploit', STEPS_POLICY)  # Experiment 1c

# Output results
for agent in agents:
    print(f"{agent.name}'s Q-table after Experiment 1c:")
    print("State  |   N       E       S       W       P       D")
    print("---------------------------------------------------")
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            q_values = agent.q_table[x, y]
            print(f"({x+1},{y+1})  |  " + "  ".join(f"{q_value:6.2f}" for q_value in q_values))
    print("\n")


red's Q-table after Experiment 1c:
State  |   N       E       S       W       P       D
---------------------------------------------------
(1,1)  |    2.26    0.00    0.00    2.66   12.43   12.24
(1,2)  |    2.16    3.06    0.00    2.12   16.45   15.28
(1,3)  |    5.17    3.46    0.00    4.22    9.25   17.66
(1,4)  |    4.30    4.81    0.00    5.18   13.55   20.31
(1,5)  |    0.97    1.95    0.00    0.00   14.77    8.20
(2,1)  |   -0.66    0.00    1.12    0.00    4.98    7.21
(2,2)  |   -0.30    0.07   -0.66   -0.17    0.00    3.90
(2,3)  |    1.26    1.41    1.02    0.57   10.03    0.00
(2,4)  |    2.98    2.95    4.01    0.99   14.46    5.40
(2,5)  |   -0.33    0.44   -0.07    0.00    4.48   10.11
(3,1)  |   -0.66    0.00   -0.30   -0.76    0.00    0.00
(3,2)  |    1.40    0.98    0.78    1.93   12.43    0.00
(3,3)  |    3.42    1.37    2.70    1.33   11.10   10.62
(3,4)  |    3.66    3.48    3.30    6.61    4.98   18.92
(3,5)  |    7.35    1.63    6.52    0.00   26.00   20.88
(4,1)

In [217]:
import numpy as np

class QTable:
    def __init__(self, num_states, num_actions, learning_rate=0.1, discount_factor=0.9):
        self.num_states = num_states
        self.num_actions = num_actions
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.q_table = np.zeros((num_states, num_actions))

    def update_q_value(self, state, action, reward, next_state):
        current_q_value = self.q_table[state, action]
        max_next_q_value = np.max(self.q_table[next_state])
        new_q_value = (1 - self.learning_rate) * current_q_value + \
                      self.learning_rate * (reward + self.discount_factor * max_next_q_value)
        self.q_table[state, action] = new_q_value

    def get_q_value(self, state, action):
        return self.q_table[state, action]

    def update_learning_rate(self, new_learning_rate):
        self.learning_rate = new_learning_rate

    def update_discount_factor(self, new_discount_factor):
        self.discount_factor = new_discount_factor

    def get_optimal_action(self, state, exploration_rate):
        if np.random.rand() < exploration_rate:
            return np.random.randint(self.num_actions)  # Randomly choose an action
        else:
            return np.argmax(self.q_table[state])  # Choose action with maximum Q-value

class BlockTransportationProblem:
    def __init__(self, num_agents, num_states, num_actions, learning_rate, discount_factor):
        self.num_agents = num_agents
        self.num_states = num_states
        self.num_actions = num_actions
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.q_tables = [QTable(num_states, num_actions, learning_rate, discount_factor) for _ in range(num_agents)]

    def update_q_values(self, agent_index, state, action, reward, next_state):
        self.q_tables[agent_index].update_q_value(state, action, reward, next_state)

    def select_action(self, agent_index, state, exploration_rate, policy):
        if policy == "PRANDOM":
            return np.random.randint(self.num_actions)  # Choose action randomly
        elif policy == "PEXPLOIT":
            return self.q_tables[agent_index].get_optimal_action(state, exploration_rate)
        elif policy == "PGREEDY":
            return np.argmax(self.q_tables[agent_index].q_table[state])  # Choose action with maximum Q-value

# Experiment parameters
num_agents = 3
num_states = 25  # Assuming 25 states
num_actions = 4  # Assuming 4 actions (north, south, east, west)
learning_rate = 0.3
discount_factor = 0.5
total_steps = 9000
initial_prandom_steps = 500
remaining_steps = total_steps - initial_prandom_steps
policy_switch_step = initial_prandom_steps

# Initialize Block Transportation Problem
btp = BlockTransportationProblem(num_agents, num_states, num_actions,learning_rate,discount_factor)

# Run PRANDOM for initial steps
for step in range(initial_prandom_steps):
    states = [np.random.randint(num_states) for _ in range(num_agents)]
    actions = [btp.select_action(agent_index, state, exploration_rate=1.0, policy="PRANDOM") for agent_index, state in enumerate(states)]
    rewards = [np.random.randint(10) for _ in range(num_agents)]  # Random rewards for demonstration
    next_states = [np.random.randint(num_states) for _ in range(num_agents)]
    for agent_index in range(num_agents):
        btp.update_q_values(agent_index, states[agent_index], actions[agent_index], rewards[agent_index], next_states[agent_index])

# Switch policies and continue running
for step in range(policy_switch_step, total_steps):
    policy = "PGREEDY" if step < (total_steps - remaining_steps // 2) else "PEXPLOIT"
    exploration_rate = 0.1 if policy == "PEXPLOIT" else 1.0
    states = [np.random.randint(num_states) for _ in range(num_agents)]
    actions = [btp.select_action(agent_index, state, exploration_rate, policy) for agent_index, state in enumerate(states)]
    rewards = [np.random.randint(10) for _ in range(num_agents)]  # Random rewards for demonstration
    next_states = [np.random.randint(num_states) for _ in range(num_agents)]
    for agent_index in range(num_agents):
        btp.update_q_values(agent_index, states[agent_index], actions[agent_index], rewards[agent_index], next_states[agent_index])

final_states = [np.random.randint(num_states) for _ in range(num_agents)]

policy = "PEXPLOIT"
final_q_table = btp.q_tables[0].q_table  # Assuming agents have the same Q-table

In [219]:
print(final_q_table)

[[ 7.00503694  7.78732465  8.59852831  8.60352702]
 [ 6.05355659  6.9877843   6.61093801  8.45854186]
 [11.69697107  6.85421518  5.51693656  7.12918729]
 [ 8.67679623  6.30499648  7.90454484  7.76605359]
 [ 6.5740658   6.14716011  6.70127726  7.89620298]
 [ 7.20960279  7.07304751  6.12619347  7.89538157]
 [ 5.71143053  7.4342129   8.68682991 10.22936457]
 [ 8.11074145  8.49385569  6.93596339  7.83008148]
 [ 7.22462064  7.54871298  7.67656901 10.45340967]
 [ 7.2511047   9.97652313  6.76792038  6.25029643]
 [ 5.11537363  9.62588791  6.95030787  6.67769375]
 [ 7.95021029 11.15103703  7.57058169  7.42208313]
 [ 6.3392159   8.66268703  6.88790682 10.38479038]
 [ 5.55388604  8.64765435  6.41312333  6.93900456]
 [ 9.64091392  6.97437254  6.79949482  6.67494951]
 [ 7.31530805  7.43805912  7.21960983  9.33020618]
 [ 7.75194522  5.0191504   8.15040355 11.2883345 ]
 [ 6.49864004  6.59574904  8.89215252  7.03100242]
 [ 9.47166001  6.83349677  7.67398663  7.73384051]
 [ 6.6392951   5.53516481  6.41

In [189]:
import numpy as np

class QTableSARSA:
    def __init__(self, num_states, num_actions, learning_rate=0.1, discount_factor=0.9):
        self.num_states = num_states
        self.num_actions = num_actions
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.q_table = np.zeros((num_states, num_actions))

    def update_q_value(self, state, action, reward, next_state, next_action):
        current_q_value = self.q_table[state, action]
        next_q_value = self.q_table[next_state, next_action]
        new_q_value = (1 - self.learning_rate) * current_q_value + \
                      self.learning_rate * (reward + self.discount_factor * next_q_value)
        self.q_table[state, action] = new_q_value

    def get_q_value(self, state, action):
        return self.q_table[state, action]

    def update_learning_rate(self, new_learning_rate):
        self.learning_rate = new_learning_rate

    def update_discount_factor(self, new_discount_factor):
        self.discount_factor = new_discount_factor

    def get_optimal_action(self, state, exploration_rate):
        if np.random.rand() < exploration_rate:
            return np.random.randint(self.num_actions)  # Randomly choose an action
        else:
            return np.argmax(self.q_table[state])  # Choose action with maximum Q-value

class BlockTransportationProblemSARSA:
    def __init__(self, num_agents, num_states, num_actions, learning_rate, discount_factor):
        self.num_agents = num_agents
        self.num_states = num_states
        self.num_actions = num_actions
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.q_tables = [QTableSARSA(num_states, num_actions, learning_rate, discount_factor) for _ in range(num_agents)]

    def update_q_values(self, agent_index, state, action, reward, next_state, next_action):
        self.q_tables[agent_index].update_q_value(state, action, reward, next_state, next_action)

    def select_action(self, agent_index, state, exploration_rate, policy):
        if policy == "PRANDOM":
            return np.random.randint(self.num_actions)  # Choose action randomly
        elif policy == "PEXPLOIT":
            return self.q_tables[agent_index].get_optimal_action(state, exploration_rate)
        elif policy == "PGREEDY":
            return np.argmax(self.q_tables[agent_index].q_table[state])  # Choose action with maximum Q-value

# Experiment parameters
num_agents = 3
num_states = 25  # Assuming 25 states
num_actions = 4  # Assuming 4 actions (north, south, east, west)
learning_rate = 0.3
discount_factor = 0.5
total_steps = 9000
initial_prandom_steps = 500
remaining_steps = total_steps - initial_prandom_steps
policy_switch_step = initial_prandom_steps

# Initialize Block Transportation Problem with SARSA
btp_sarsa = BlockTransportationProblemSARSA(num_agents, num_states, num_actions, learning_rate, discount_factor)

# Run PRANDOM for initial steps
for step in range(initial_prandom_steps):
    states = [np.random.randint(num_states) for _ in range(num_agents)]
    actions = [btp_sarsa.select_action(agent_index, state, exploration_rate=1.0, policy="PRANDOM") for agent_index, state in enumerate(states)]
    rewards = [np.random.randint(10) for _ in range(num_agents)]  # Random rewards for demonstration
    next_states = [np.random.randint(num_states) for _ in range(num_agents)]
    next_actions = [btp_sarsa.select_action(agent_index, next_state, exploration_rate=0.1, policy="PEXPLOIT") for agent_index, next_state in enumerate(next_states)]
    for agent_index in range(num_agents):
        btp_sarsa.update_q_values(agent_index, states[agent_index], actions[agent_index], rewards[agent_index], next_states[agent_index], next_actions[agent_index])

# Switch policies and continue running
for step in range(policy_switch_step, total_steps):
    policy = "PGREEDY" if step < (total_steps - remaining_steps // 2) else "PEXPLOIT"
    exploration_rate = 0.1 if policy == "PEXPLOIT" else 1.0
    states = [np.random.randint(num_states) for _ in range(num_agents)]
    actions = [btp_sarsa.select_action(agent_index, state, exploration_rate, policy) for agent_index, state in enumerate(states)]
    rewards = [np.random.randint(10) for _ in range(num_agents)]  # Random rewards for demonstration
    next_states = [np.random.randint(num_states) for _ in range(num_agents)]
    next_actions = [btp_sarsa.select_action(agent_index, next_state, exploration_rate, policy) for agent_index, next_state in enumerate(next_states)]
    for agent_index in range(num_agents):
        btp_sarsa.update_q_values(agent_index, states[agent_index], actions[agent_index], rewards[agent_index], next_states[agent_index], next_actions[agent_index])

# Report one of the final Q-tables
final_q_table_sarsa = btp_sarsa.q_tables[0].q_table  # Assuming agents have the same Q-table
print("Final Q-Table for Experiment 2 (SARSA):")
print(final_q_table_sarsa)

# Assess the quality of agent coordination
# You can assess coordination by analyzing metrics such as total rewards, convergence speed, and task completion rates.


Final Q-Table for Experiment 2 (SARSA):
[[ 7.69489145  7.30731207  9.15917982  7.20759023]
 [ 7.70551939  7.83009582 10.18136422  7.69689014]
 [ 6.43137058  5.36012178  6.84581668  9.67628003]
 [ 6.92972656  7.88601297  6.96209785  9.64069805]
 [ 5.91817267  7.74922127  6.53013358  7.04838564]
 [ 7.17631581  7.85516956  6.86019335  8.90321939]
 [11.49593275  7.18641286  6.91201564  9.00027687]
 [ 7.21555706  7.23357596  8.13285405  6.9126108 ]
 [ 7.56162342  9.21199453  7.48434565  6.56246388]
 [ 7.51717167  6.8564807  10.24051227  6.74161605]
 [ 8.06489509  5.82176862  7.33390791  6.44948564]
 [ 6.04554829  9.9207904   5.757327    5.67044394]
 [ 8.55599842  7.77909734  6.57297255  6.0612052 ]
 [ 7.14260543  6.63142825  6.90321385  9.29520028]
 [ 8.00602835  7.52976584  6.29494984  6.65833033]
 [ 6.42009631  9.19182138  6.48752771  7.14018259]
 [ 6.31456673 10.28249488  6.02792077  7.12299293]
 [ 6.07957032  5.58712984  8.43679549  9.65963799]
 [ 6.40197019  7.41625154  7.87675334 10.2

<h3> EXP2  SARSA Q-learning 9000 (500,8500) &report  Q-tables  </h3>

<h3> EXP3 From 1.c (Qlearning) Different Learning rate (0.15,0.45)  </h3>

In [4]:
import numpy as np
import random

# Define the environment setup
GRID_SIZE = 5
PICKUP_LOCATIONS = {(0, 4): 5, (1, 3): 5, (4, 1): 5}
DROPOFF_LOCATIONS = {(0, 0): 0, (2, 0): 0, (3, 4): 0}
ACTIONS = ['N', 'E', 'S', 'W', 'P', 'D']
ACTION_REWARDS = {'P': 13, 'D': 13, 'N': -1, 'E': -1, 'S': -1, 'W': -1}

# Hyperparameters
LEARNING_RATE = 0.15
DISCOUNT_FACTOR = 0.5
STEPS_PRANDOM = 500  # Initial random steps
STEPS_POLICY = 8500  # Steps for PExploit policy

# Seed for reproducibility
random.seed(4)
np.random.seed(4)

class Agent:
    def __init__(self, start_position, name):
        self.position = start_position
        self.name = name
        self.carrying = False
        self.q_table = np.zeros((GRID_SIZE, GRID_SIZE, len(ACTIONS)))

    def get_valid_actions(self):
        x, y = self.position
        valid_actions = ACTIONS.copy()
        if x == 0: valid_actions.remove('N')
        if x == GRID_SIZE - 1: valid_actions.remove('S')
        if y == 0: valid_actions.remove('W')
        if y == GRID_SIZE - 1: valid_actions.remove('E')
        if self.carrying and (x, y) in DROPOFF_LOCATIONS:
            valid_actions.append('D')
        elif not self.carrying and (x, y) in PICKUP_LOCATIONS and PICKUP_LOCATIONS[(x, y)] > 0:
            valid_actions.append('P')
        return valid_actions

    def select_action(self, valid_actions, policy):
        if not valid_actions:
            return None
        if policy == 'PRandom':
            return random.choice(valid_actions)
        elif policy == 'PExploit':
            q_values = [self.q_table[x, y, ACTIONS.index(a)] for a in valid_actions]
            max_value = max(q_values)
            best_actions = [a for a, q in zip(valid_actions, q_values) if q == max_value]
            return random.choice(best_actions)

    def perform_action(self, action):
        if action is None:
            return
        move_delta = {'N': (-1, 0), 'S': (1, 0), 'E': (0, 1), 'W': (0, -1)}
        x, y = self.position
        x += move_delta.get(action, (0, 0))[0]
        y += move_delta.get(action, (0, 0))[1]
        if action == 'P' and (x, y) in PICKUP_LOCATIONS:
            self.carrying = True
            PICKUP_LOCATIONS[(x, y)] -= 1
        elif action == 'D' and (x, y) in DROPOFF_LOCATIONS:
            self.carrying = False
            DROPOFF_LOCATIONS[(x, y)] += 1
        self.position = (x, y)

    def update_q_table(self, action, reward, next_position):
        if action is None:
            return
        old_q = self.q_table[self.position[0], self.position[1], ACTIONS.index(action)]
        next_max_q = np.max(self.q_table[next_position[0], next_position[1]])
        self.q_table[self.position[0], self.position[1], ACTIONS.index(action)] = \
            (1 - LEARNING_RATE) * old_q + LEARNING_RATE * (reward + DISCOUNT_FACTOR * next_max_q)

    def is_terminal_state(self):
        return all(value == 5 for value in DROPOFF_LOCATIONS.values())

    def reset(self):
        self.position = initial_positions[self.name]
        self.carrying = False
        for loc in PICKUP_LOCATIONS:
            PICKUP_LOCATIONS[loc] = 5
        for loc in DROPOFF_LOCATIONS:
            DROPOFF_LOCATIONS[loc] = 0

initial_positions = {'red': (2, 2), 'blue': (4, 2), 'black': (0, 2)}
agents = [Agent(initial_positions[name], name) for name in initial_positions]

def run_experiment(agents, policy, steps):
    for step in range(steps):
        for agent in agents:
            valid_actions = agent.get_valid_actions()
            action = agent.select_action(valid_actions, policy)
            agent.perform_action(action)
            reward = ACTION_REWARDS.get(action, 0)
            agent.update_q_table(action, reward, agent.position)
            if agent.is_terminal_state():
                agent.reset()

# Run Experiments
run_experiment(agents, 'PRandom', STEPS_PRANDOM)  # Initial random phase
run_experiment(agents, 'PExploit', STEPS_POLICY)  # Experiment 1c

# Output results
for agent in agents:
    print(f"{agent.name}'s Q-table after Experiment 1c:")
    print("State  |   N       E       S       W       P       D")
    print("---------------------------------------------------")
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            q_values = agent.q_table[x, y]
            print(f"({x+1},{y+1})  |  " + "  ".join(f"{q_value:6.2f}" for q_value in q_values))
    print("\n")


red's Q-table after Experiment 1c:
State  |   N       E       S       W       P       D
---------------------------------------------------
(1,1)  |    0.48    0.00    0.00    0.69    6.97    6.19
(1,2)  |    0.49    0.81    0.00    0.32    9.81    8.11
(1,3)  |    1.64    0.89    0.00    1.26    4.37   10.94
(1,4)  |    1.14    1.48    0.00    1.63    7.32   13.27
(1,5)  |    0.04    0.30    0.00    0.00    8.50    4.02
(2,1)  |   -0.39    0.00    0.13    0.00    2.23    3.75
(2,2)  |   -0.15   -0.13   -0.39   -0.33    0.00    1.95
(2,3)  |    0.15    0.04    0.03    0.00    5.42    0.00
(2,4)  |    0.74    0.69    1.13    0.13    8.39    2.36
(2,5)  |   -0.53   -0.10   -0.24    0.00    2.10    5.43
(3,1)  |   -0.39    0.00   -0.15   -0.48    0.00    0.00
(3,2)  |    0.25    0.13    0.13    0.29    6.97    0.00
(3,3)  |    0.77    0.16    0.62    0.24    5.77    5.57
(3,4)  |    0.98    0.88    0.83    2.49    2.23   12.06
(3,5)  |    2.87    0.09    2.30    0.00   26.00   14.08
(4,1)

In [5]:
import numpy as np
import random

# Define the environment setup
GRID_SIZE = 5
PICKUP_LOCATIONS = {(0, 4): 5, (1, 3): 5, (4, 1): 5}
DROPOFF_LOCATIONS = {(0, 0): 0, (2, 0): 0, (3, 4): 0}
ACTIONS = ['N', 'E', 'S', 'W', 'P', 'D']
ACTION_REWARDS = {'P': 13, 'D': 13, 'N': -1, 'E': -1, 'S': -1, 'W': -1}

# Hyperparameters
LEARNING_RATE = 0.45
DISCOUNT_FACTOR = 0.5
STEPS_PRANDOM = 500  # Initial random steps
STEPS_POLICY = 8500  # Steps for PExploit policy

# Seed for reproducibility
random.seed(4)
np.random.seed(4)

class Agent:
    def __init__(self, start_position, name):
        self.position = start_position
        self.name = name
        self.carrying = False
        self.q_table = np.zeros((GRID_SIZE, GRID_SIZE, len(ACTIONS)))

    def get_valid_actions(self):
        x, y = self.position
        valid_actions = ACTIONS.copy()
        if x == 0: valid_actions.remove('N')
        if x == GRID_SIZE - 1: valid_actions.remove('S')
        if y == 0: valid_actions.remove('W')
        if y == GRID_SIZE - 1: valid_actions.remove('E')
        if self.carrying and (x, y) in DROPOFF_LOCATIONS:
            valid_actions.append('D')
        elif not self.carrying and (x, y) in PICKUP_LOCATIONS and PICKUP_LOCATIONS[(x, y)] > 0:
            valid_actions.append('P')
        return valid_actions

    def select_action(self, valid_actions, policy):
        if not valid_actions:
            return None
        if policy == 'PRandom':
            return random.choice(valid_actions)
        elif policy == 'PExploit':
            q_values = [self.q_table[x, y, ACTIONS.index(a)] for a in valid_actions]
            max_value = max(q_values)
            best_actions = [a for a, q in zip(valid_actions, q_values) if q == max_value]
            return random.choice(best_actions)

    def perform_action(self, action):
        if action is None:
            return
        move_delta = {'N': (-1, 0), 'S': (1, 0), 'E': (0, 1), 'W': (0, -1)}
        x, y = self.position
        x += move_delta.get(action, (0, 0))[0]
        y += move_delta.get(action, (0, 0))[1]
        if action == 'P' and (x, y) in PICKUP_LOCATIONS:
            self.carrying = True
            PICKUP_LOCATIONS[(x, y)] -= 1
        elif action == 'D' and (x, y) in DROPOFF_LOCATIONS:
            self.carrying = False
            DROPOFF_LOCATIONS[(x, y)] += 1
        self.position = (x, y)

    def update_q_table(self, action, reward, next_position):
        if action is None:
            return
        old_q = self.q_table[self.position[0], self.position[1], ACTIONS.index(action)]
        next_max_q = np.max(self.q_table[next_position[0], next_position[1]])
        self.q_table[self.position[0], self.position[1], ACTIONS.index(action)] = \
            (1 - LEARNING_RATE) * old_q + LEARNING_RATE * (reward + DISCOUNT_FACTOR * next_max_q)

    def is_terminal_state(self):
        return all(value == 5 for value in DROPOFF_LOCATIONS.values())

    def reset(self):
        self.position = initial_positions[self.name]
        self.carrying = False
        for loc in PICKUP_LOCATIONS:
            PICKUP_LOCATIONS[loc] = 5
        for loc in DROPOFF_LOCATIONS:
            DROPOFF_LOCATIONS[loc] = 0

initial_positions = {'red': (2, 2), 'blue': (4, 2), 'black': (0, 2)}
agents = [Agent(initial_positions[name], name) for name in initial_positions]

def run_experiment(agents, policy, steps):
    for step in range(steps):
        for agent in agents:
            valid_actions = agent.get_valid_actions()
            action = agent.select_action(valid_actions, policy)
            agent.perform_action(action)
            reward = ACTION_REWARDS.get(action, 0)
            agent.update_q_table(action, reward, agent.position)
            if agent.is_terminal_state():
                agent.reset()

# Run Experiments
run_experiment(agents, 'PRandom', STEPS_PRANDOM)  # Initial random phase
run_experiment(agents, 'PExploit', STEPS_POLICY)  # Experiment 1c

# Output results
for agent in agents:
    print(f"{agent.name}'s Q-table after Experiment 1c:")
    print("State  |   N       E       S       W       P       D")
    print("---------------------------------------------------")
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            q_values = agent.q_table[x, y]
            print(f"({x+1},{y+1})  |  " + "  ".join(f"{q_value:6.2f}" for q_value in q_values))
    print("\n")


red's Q-table after Experiment 1c:
State  |   N       E       S       W       P       D
---------------------------------------------------
(1,1)  |    4.63    0.00    0.00    5.10   16.62   17.34
(1,2)  |    4.42    5.79    0.00    4.57   20.73   20.45
(1,3)  |    8.29    6.33    0.00    7.26   14.09   21.63
(1,4)  |    7.60    8.05    0.00    8.39   18.47   23.74
(1,5)  |    2.62    4.30    0.00    0.00   19.21   12.37
(2,1)  |   -0.83    0.00    2.62    0.00    8.19   10.38
(2,2)  |   -0.45    0.62   -0.83    0.41    0.00    5.85
(2,3)  |    3.02    3.25    2.55    1.64   13.90    0.00
(2,4)  |    5.35    5.64    6.87    2.43   18.73    8.98
(2,5)  |    0.33    1.57    0.48    0.00    7.17   14.13
(3,1)  |   -0.83    0.00   -0.45   -0.91    0.00    0.00
(3,2)  |    3.15    2.36    1.89    4.13   16.62    0.00
(3,3)  |    6.24    3.31    5.34    2.92   15.68   15.09
(3,4)  |    6.55    6.30    5.90    9.39    8.19   22.62
(3,5)  |   10.16    3.56    9.55    0.00   26.00   23.97
(4,1)

(Discussion) Analyzing the effects of using the 3 different learning rates on the system performance 

<h3> EXP4 From 1c alpha =0.3, Gamma=0.5  ,Q-learning three pickup locations to: (4,2), (3,3) and (2,4)</h3>

In [ ]:
import numpy as np
import random

# Define the environment setup
GRID_SIZE = 5
PICKUP_LOCATIONS = {(0, 4): 5, (1, 3): 5, (4, 1): 5}
DROPOFF_LOCATIONS = {(0, 0): 0, (2, 0): 0, (3, 4): 0}
ACTIONS = ['N', 'E', 'S', 'W', 'P', 'D']
ACTION_REWARDS = {'P': 13, 'D': 13, 'N': -1, 'E': -1, 'S': -1, 'W': -1}

# Hyperparameters
LEARNING_RATE = 0.45
DISCOUNT_FACTOR = 0.5
STEPS_PRANDOM = 500  # Initial random steps
STEPS_POLICY = 8500  # Steps for PExploit policy

# Seed for reproducibility
random.seed(4)
np.random.seed(4)

class Agent:
    def __init__(self, start_position, name):
        self.position = start_position
        self.name = name
        self.carrying = False
        self.q_table = np.zeros((GRID_SIZE, GRID_SIZE, len(ACTIONS)))

    def get_valid_actions(self):
        x, y = self.position
        valid_actions = ACTIONS.copy()
        if x == 0: valid_actions.remove('N')
        if x == GRID_SIZE - 1: valid_actions.remove('S')
        if y == 0: valid_actions.remove('W')
        if y == GRID_SIZE - 1: valid_actions.remove('E')
        if self.carrying and (x, y) in DROPOFF_LOCATIONS:
            valid_actions.append('D')
        elif not self.carrying and (x, y) in PICKUP_LOCATIONS and PICKUP_LOCATIONS[(x, y)] > 0:
            valid_actions.append('P')
        return valid_actions

    def select_action(self, valid_actions, policy):
        if not valid_actions:
            return None
        if policy == 'PRandom':
            return random.choice(valid_actions)
        elif policy == 'PExploit':
            q_values = [self.q_table[x, y, ACTIONS.index(a)] for a in valid_actions]
            max_value = max(q_values)
            best_actions = [a for a, q in zip(valid_actions, q_values) if q == max_value]
            return random.choice(best_actions)

    def perform_action(self, action):
        if action is None:
            return
        move_delta = {'N': (-1, 0), 'S': (1, 0), 'E': (0, 1), 'W': (0, -1)}
        x, y = self.position
        x += move_delta.get(action, (0, 0))[0]
        y += move_delta.get(action, (0, 0))[1]
        if action == 'P' and (x, y) in PICKUP_LOCATIONS:
            self.carrying = True
            PICKUP_LOCATIONS[(x, y)] -= 1
        elif action == 'D' and (x, y) in DROPOFF_LOCATIONS:
            self.carrying = False
            DROPOFF_LOCATIONS[(x, y)] += 1
        self.position = (x, y)

    def update_q_table(self, action, reward, next_position):
        if action is None:
            return
        old_q = self.q_table[self.position[0], self.position[1], ACTIONS.index(action)]
        next_max_q = np.max(self.q_table[next_position[0], next_position[1]])
        self.q_table[self.position[0], self.position[1], ACTIONS.index(action)] = \
            (1 - LEARNING_RATE) * old_q + LEARNING_RATE * (reward + DISCOUNT_FACTOR * next_max_q)

    def is_terminal_state(self):
        return all(value == 5 for value in DROPOFF_LOCATIONS.values())

    def reset(self):
        self.position = initial_positions[self.name]
        self.carrying = False
        for loc in PICKUP_LOCATIONS:
            PICKUP_LOCATIONS[loc] = 5
        for loc in DROPOFF_LOCATIONS:
            DROPOFF_LOCATIONS[loc] = 0

initial_positions = {'red': (2, 2), 'blue': (4, 2), 'black': (0, 2)}
agents = [Agent(initial_positions[name], name) for name in initial_positions]

def run_experiment(agents, policy, steps):
    for step in range(steps):
        for agent in agents:
            valid_actions = agent.get_valid_actions()
            action = agent.select_action(valid_actions, policy)
            agent.perform_action(action)
            reward = ACTION_REWARDS.get(action, 0)
            agent.update_q_table(action, reward, agent.position)
            if agent.is_terminal_state():
                agent.reset()

# Run Experiments
run_experiment(agents, 'PRandom', STEPS_PRANDOM)  # Initial random phase
run_experiment(agents, 'PExploit', STEPS_POLICY)  # Experiment 1c

# Output results
for agent in agents:
    print(f"{agent.name}'s Q-table after Experiment 1c:")
    print("State  |   N       E       S       W       P       D")
    print("---------------------------------------------------")
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            q_values = agent.q_table[x, y]
            print(f"({x+1},{y+1})  |  " + "  ".join(f"{q_value:6.2f}" for q_value in q_values))
    print("\n")


 Discussion))) When interpreting the results of this experiment center on analyzing on how well the learning strategy was able to adapt to the change of the pickup locations and to which extend it was able to learn “new” paths and unlearn “old” paths which became obsolete

note:For all EXPERMINENTS, if a terminal state is reached, restart the experiment by resetting the PD world to the initial state, but do not reset the Q-table. Run each experiment twice, 

Report  and interpret the results; e.g., utilities computed, rewards obtained in various stages of each experiment. 

Assess which experiment obtained the best results . Next, analyze the various q-tables you created and try identify attractive paths  in the obtained q-tables, if there are any. Moreover, briefly assess if your system gets better after it solved a few PD-world problems—reached the terminal state at least once. Briefly analyze to which extend the results of the two different runs agree and disagree in the 4 experiments. Analyze agent coordination for experiments 1.c and 4. Finally, analyze how well the approach adapted to change in the fourth experiment
